In [5]:
import pandas as pd
import networkx as nx
from torch_geometric.data import Data
import torch

import warnings
warnings.filterwarnings("ignore")

In [15]:
data = pd.read_csv('../data/processed/main_data.csv')

In [16]:
data.head(5)

,customer_id,customer_city,customer_state,order_purchase_timestamp,product_id,price,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,review_score
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP,2017-05-16,a9516a079e37a9c9c36b9b78b10169e8,124.99,8683.0,54.0,64.0,31.0,office_furniture,4
1,18955e83d337fd6b2def6b18a428ac77,sao bernardo do campo,SP,2018-01-12,4aa6014eceb682077f9dc4bffebc05b0,289.00,10150.0,89.0,15.0,40.0,housewares,5
2,4e7b3e00288586ebd08712fdd0374a03,sao paulo,SP,2018-05-19,bd07b66896d6f1494f5b86251848ced7,139.94,8267.0,52.0,52.0,17.0,office_furniture,5
3,b2b6027bc5c5109e529d4dc6358b12c3,mogi das cruzes,SP,2018-03-13,a5647c44af977b148e0a3a4751a09e2e,149.94,12160.0,56.0,51.0,28.0,office_furniture,5
4,4f2d8ab171c80ec8364f7c12e35b23ad,campinas,SP,2018-07-29,9391a573abe00141c56e38d84d7d5b3b,230.00,5200.0,45.0,15.0,35.0,home_confort,5


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110750 entries, 0 to 110749
Data columns (total 12 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    110750 non-null  object 
 1   customer_city                  110750 non-null  object 
 2   customer_state                 110750 non-null  object 
 3   order_purchase_timestamp       110750 non-null  object 
 4   product_id                     110750 non-null  object 
 5   price                          110750 non-null  float64
 6   product_weight_g               110749 non-null  float64
 7   product_length_cm              110749 non-null  float64
 8   product_height_cm              110749 non-null  float64
 9   product_width_cm               110749 non-null  float64
 10  product_category_name_english  110750 non-null  object 
 11  review_score                   110750 non-null  int64  
dtypes: float64(5), int64(1), objec

In [18]:
graph = nx.Graph()

In [19]:
for customer in data['customer_id'].unique():
    graph.add_node(customer, type='customer')

In [20]:
for product in data['product_id'].unique():
    graph.add_node(product, type='product')

In [21]:
for _, row in data.iterrows():
    graph.add_edge(row['customer_id'], row['product_id'])

In [22]:
node_mapping = {node: i for i, node in enumerate(graph.nodes)}

In [23]:
node_features = torch.zeros(len(graph.nodes), 1)

In [24]:
edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges], dtype=torch.long).t().contiguous()

In [25]:
data = Data(x=node_features, edge_index=edge_index)

In [29]:
print("Number of nodes:", graph.number_of_nodes())
print("Number of edges:", graph.number_of_edges())

Number of nodes: 128688
Number of edges: 100158


In [27]:
torch.save(data, '../data/processed/graph_data.pt')